## ВШЭ, ФКН, Программа "Специалист по Data Science" (2021/2022)
## Курс "Машинное обучение"
## ДЗ №0: Введение в ML
## Выполнил: Кирилл Н., ibnkir@yandex.ru, 09.12.2021 г.
***

## Часть 1 - Реализация простейшего kNN-классификатора

*Примечание: Часть 2 "Анализ данных по сердечно-сосудистым заболеваниям" - см. [ниже](#part2)*

In [45]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale

In [46]:
df_titanic = pd.read_csv('titanic.csv')
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
df_titanic.shape

(891, 12)

In [48]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


> Обратите внимание на разное количество значений переменных. Если это так, то у вас пропуски в данных. Такое бывает. 
Например, в этом датасете не получилось выяснить всех подробностей у некоторых людей ввиду утери данных, невозможности узнать или восстановить их.

In [49]:
df_titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Заполните пропущенные значения чем-нибудь.**

На будущее - в sklearn есть такой модуль - http://scikit-learn.org/stable/modules/impute.html, но сейчас можете заполнить пропуски средствами pandas - метод fillna()

In [50]:
# Заполняем пропуски для признака Age (заменяем NaN на медианное значение по всей выборке)
df_titanic['Age'].fillna(value=df_titanic['Age'].median(), inplace=True)

**Заведите бинарный признак is_man, отвечающий за пол. Строковый удалите.**

Хинт - есть метод apply()

In [51]:
# Добавляем бинарный признак is_man и удаляем строковый столбец Sex
df_titanic['is_man'] = df_titanic['Sex'].apply(lambda x: 1 if x=='male' else 0)
df_titanic.drop('Sex', axis=1, inplace=True)

**Сформируйте трейн и тест выборку, где в X лежат признаки 'is_man', 'Age', 'Pclass', а в у - Survived.**

Результат - numpy array.

In [52]:
x_columns = ['is_man', 'Age', 'Pclass']

X_train = df_titanic[x_columns].loc[:500].to_numpy()
y_train = df_titanic['Survived'].loc[:500].to_numpy()

X_test = df_titanic[x_columns].loc[500:].to_numpy()
y_test = df_titanic['Survived'].loc[500:].to_numpy()

In [53]:
assert isinstance(X_train, np.ndarray)
assert isinstance(X_test, np.ndarray)
assert isinstance(y_train, np.ndarray)
assert isinstance(y_test, np.ndarray)

In [54]:
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

In [55]:
assert not np.any(np.isnan(X_train)), 'у вас в данных есть пропущенные значения. Заполните их или удалите'
assert not np.any(np.isnan(X_test)), 'у вас в данных есть пропущенные значения. Заполните их или удалите'

**Реализуйте kNN cами, как делали это на лекции.**

Можете улучшить его (например, использовать kd-tree для быстрого поиска ближайших соседей, или обходить соседей и класть расстояния в мин-кучу - тогда вы быстрее будете набирать их, без сортировки).

In [56]:
# Простейший вариант классификатора
class MyKNNClassifier:
    """
    simple KNN classifier with O(n) complexity for memory,
    and O(n log n) complexity - for nn search.
    """
        
    def __init__(self, n):
        self.n = n
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        
    def predict(self, X):
        y_pred = []
        
        for i in range(len(X)):
            dist = [np.dot(X[i]-self.X[j], X[i]-self.X[j]) for j in range(len(self.X))]
            idx = np.argsort(dist)[:self.n]
            counts = self.y[idx]
            y_pred.append(np.mean(counts) > 0.5)
            
        return(np.array(y_pred))

In [57]:
my_knn_clf = MyKNNClassifier(5)
my_knn_clf.fit(X_train, y_train)
y_pred = my_knn_clf.predict(X_test)
score = np.mean(y_pred == y_test)
print('Показатель точности: {:.4f}'.format(score))

Показатель точности: 0.7877


> Мы получили бейслайн решение. Оно должно давать score в районе 0.76

In [58]:
# Проверим точность нашего классификатора с помощью sklearn
score = accuracy_score(y_test, y_pred)
print('Показатель точности: {:.4f}'.format(score))

Показатель точности: 0.7877


In [59]:
# Используем класификатор из библиотеки sklearn
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
score = accuracy_score(y_test, y_pred)
print('Показатель точности классификатора из sklearn: {:.4f}'.format(score))

Показатель точности классификатора из sklearn: 0.7698


**Добавьте еще несколько признаков (как минимум, Cabin, Fare и Embarked) и предобработайте их для нашего алгоритма. Помним, что он метрический и основан на том, что мы будем считать расстояния. 
Для этого нужно подготовить данные, т.е. привести к одному масштабу и т.д.**

https://sebastianraschka.com/Articles/2014_about_feature_scaling.html#about-standardization

In [60]:
# Масштабируем признаки Fare и Age
df_titanic['Fare_minmax'] = (df_titanic['Fare'] - df_titanic['Fare'].min()) / \
                                (df_titanic['Fare'].max() - df_titanic['Fare'].min())

df_titanic['Age_minmax'] = (df_titanic['Age'] - df_titanic['Age'].min()) / \
                                (df_titanic['Age'].max() - df_titanic['Age'].min())

**Далее нужно заполнить пропуски.**

Н-р, из признака Cabin можно извлечь тип палубы.

In [61]:
# Добавляем столбец Deck (тип палубы - соответствует 1-й букве в номере кабины) и заполняем пропуски с помощью столбцов 
# Cabin и Pclass, т.е. вместо NaN вставляем наиболее часто встречающуюся палубу для данного класса Pclass

df_titanic['Deck'] = df_titanic[~df_titanic['Cabin'].isnull()]['Cabin'].apply(lambda x: x[0])

from collections import Counter

pclass1_decks = df_titanic[(df_titanic['Pclass']==1) & (~df_titanic['Deck'].isnull())]['Deck'].values
pclass1_most_common_deck = Counter(pclass1_decks).most_common()[0][0]

pclass2_decks = df_titanic[(df_titanic['Pclass']==2) & (~df_titanic['Deck'].isnull())]['Deck'].values
pclass2_most_common_deck = Counter(pclass2_decks).most_common()[0][0]

pclass3_decks = df_titanic[(df_titanic['Pclass']==3) & (~df_titanic['Deck'].isnull())]['Deck'].values
pclass3_most_common_deck = Counter(pclass3_decks).most_common()[0][0]

''' 
# Вариант заполнения пропусков с помощью циклов
for i in range(len(df_titanic)): 
    if not pd.isnull(df_titanic.loc[i, 'Deck']):
        continue
    
    if df_titanic.loc[i, 'Pclass']==1:
        df_titanic.loc[i,'Deck'] = pclass1_most_common_deck
    elif df_titanic.loc[i,'Pclass']==2:
        df_titanic.loc[i, 'Deck'] = pclass2_most_common_deck
    else:
        df_titanic.loc[i, 'Deck'] = pclass3_most_common_deck    
'''

# Вариант заполнения пропусков с помощью словаря
freq_dict = {
    1: pclass1_most_common_deck,
    2: pclass2_most_common_deck,
    3: pclass3_most_common_deck
}

df_titanic['Deck'] = df_titanic.apply(
    lambda row: freq_dict[row['Pclass']] if pd.isnull(row['Deck']) else row['Deck'],
    axis=1
)

# Есть еще 3-й вариант заполнения пропусков - сформировать колонку с частотами, 
# после чего выполнить df['Deck'].fillna(df_titanic.Frequencies)

In [62]:
# Заполняем пропуски для признака Embarked (заменяем NaN на наиболее часто встречающийся порт отправления)
ports = df_titanic[~df_titanic['Embarked'].isnull()]['Embarked'].values
most_common_port = Counter(ports).most_common()[0][0]
df_titanic['Embarked'].fillna(value=most_common_port, inplace=True)

**Признаки бывают непрерывными и категориальными. К категориальным признакам можно отнести например строки, тип палубы и прочее.
Как их предобрабатывать, зависит от алгоритма. Самый простой способ - one-hot encoding.**

In [63]:
# Выполняем one-hot encoding для признака Embarked
df_titanic['Embarked_C'] = df_titanic['Embarked'].apply(lambda x: 1 if x=='C' else 0)
df_titanic['Embarked_Q'] = df_titanic['Embarked'].apply(lambda x: 1 if x=='Q' else 0)
df_titanic['Embarked_S'] = df_titanic['Embarked'].apply(lambda x: 1 if x=='S' else 0)

In [64]:
# Выполняем one-hot encoding для признака Deck

# Сначала выводим все возможные значения этого признака
print(np.unique(df_titanic['Deck']))

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T']


In [65]:
# Выполняем one-hot encoding для признака Deck (продолжение)
df_titanic['Deck_A'] = df_titanic['Deck'].apply(lambda x: 1 if x=='A' else 0)
df_titanic['Deck_B'] = df_titanic['Deck'].apply(lambda x: 1 if x=='B' else 0)
df_titanic['Deck_C'] = df_titanic['Deck'].apply(lambda x: 1 if x=='C' else 0)
df_titanic['Deck_D'] = df_titanic['Deck'].apply(lambda x: 1 if x=='D' else 0)
df_titanic['Deck_E'] = df_titanic['Deck'].apply(lambda x: 1 if x=='E' else 0)
df_titanic['Deck_F'] = df_titanic['Deck'].apply(lambda x: 1 if x=='F' else 0)
df_titanic['Deck_G'] = df_titanic['Deck'].apply(lambda x: 1 if x=='G' else 0)
df_titanic['Deck_T'] = df_titanic['Deck'].apply(lambda x: 1 if x=='T' else 0)

In [66]:
x_columns = ['is_man', 'Age_minmax', 'Pclass', 'Fare_minmax', \
             'Embarked_C', 'Embarked_Q', 'Embarked_S', \
             'Deck_A', 'Deck_B', 'Deck_C','Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T']

assert not np.any(np.isnan(df_titanic[x_columns])), 'у вас в данных есть пропущенные значения. Заполните их или удалите'

**Попробуйте разбить данные на три части (трейн-валидацию-тест) и подобрать гиперпараметр - число ближайших соседей. 
Как вариант, можно выбрать на валидации алгоритм с наилучшим score, а затем посмотреть его качество на тесте.**

In [69]:
# Использование кросс-валидации
max_score = 0
max_score_k = 0

# Отводим на тестовую выборку 200 последних строк
X_test = df_titanic[x_columns].loc[df_titanic.shape[0]-200:].to_numpy()
y_test = df_titanic['Survived'].loc[df_titanic.shape[0]-200:].to_numpy()

n_splits = 5 # Параметр кросс-валидации
valid_rows_num = round((df_titanic.shape[0]-200) / n_splits) # Кол-во строк в каждом валидационном блоке 

for k in range(1, 5): # Кол-во ближайших соседей
    for n in range(0, n_splits):
        X_valid = df_titanic[x_columns].loc[n*valid_rows_num : (n+1)*valid_rows_num].to_numpy()
        y_valid = df_titanic['Survived'].loc[n*valid_rows_num : (n+1)*valid_rows_num].to_numpy()
        
        idx = [i for i in range(n*valid_rows_num, (n+1)*valid_rows_num)]
        X_train = df_titanic[x_columns].drop(index=idx).to_numpy()
        y_train = df_titanic['Survived'].drop(index=idx).to_numpy()
    
        my_knn_clf = MyKNNClassifier(k)
        my_knn_clf.fit(X_train, y_train)
        y_pred = my_knn_clf.predict(X_valid)
        score = np.mean(y_pred == y_valid)
        if score >= max_score:
            max_score = score
            max_score_k = k

# Проверяем точность на тестовой выборке с найденным кол-вом соседей max_score_k
X_train = df_titanic[x_columns].loc[:df_titanic.shape[0]-200].to_numpy()
y_train = df_titanic['Survived'].loc[:df_titanic.shape[0]-200].to_numpy()

my_knn_clf = MyKNNClassifier(max_score_k)
my_knn_clf.fit(X_train, y_train)
y_pred = my_knn_clf.predict(X_test)
score = np.mean(y_pred == y_test)
print('Наибольшая точность нашего классификатора: {:.4f}, оптимальное кол-во соседей k: {}'.format(score, max_score_k))   

Наибольшая точность нашего классификатора: 0.8250, оптимальное кол-во соседей k: 4


In [70]:
# Проверим на классификаторе из sklearn
X = scale(df_titanic[x_columns].to_numpy())
y = df_titanic['Survived'].to_numpy()

kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=max_score_k)

score = np.mean(cross_val_score(knn_clf, X, y, cv=kf, scoring='accuracy'))
print('Наибольшая точность классификатора из sklearn: {:.4f}'.format(score))

Наибольшая точность классификатора из sklearn: 0.8081


## Часть 2 - Анализ данных по сердечно-сосудистым заболеваниям <a id="part2"></a>

**В задании предлагается с помощью Pandas ответить на несколько вопросов по данным о сердечно-сосудистых заболеваниях (качать данные не надо, они уже есть в репозитории). Данные использовались в соревновании [Ml Boot Camp 5](http://mlbootcamp.ru/round/12/sandbox/).**

**Заполните код в клетках (где написано "Ваш код здесь") и ответьте на вопросы в [веб-форме](https://docs.google.com/forms/d/1uzbxvBtVjj62oysHGK08Ic5mYtHzyQqUdLN7kwqT12w).**

В соревновании предлагалось определить наличие/отсутствие сердечно-сосудистых заболеваний (ССЗ) по результатам осмотра пациента.

**Описание данных.**

Объективные признаки:

 - Возраст (age)
 - Рост (height)
 - Вес (weight)
 - Пол (gender)
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее (ap_hi, ap_lo)
 - Холестерин (cholesterol)
 - Глюкоза (gluc)
 

Субъективные признаки (со слов пациентов):

 - Курение (smoke)
 - Употребление алкоголя (alco)
 - Физическая активность (active)
 
Целевой признак (который интересно будет прогнозировать):
 - Наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра (cardio)

Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Мы будем работать только с обучающей выборкой и с помощью `Pandas` проведем первичный анализ данных. 

In [34]:
import numpy as np
import pandas as pd

In [35]:
df = pd.read_csv('mlbootcamp5_train.csv', sep=';', index_col='id')
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


**<font color='red'>Вопрос 1 (1 балл).</font> Сколько мужчин и женщин представлено в этом наборе данных? Не было дано расшифровки признака "пол" (какому полу соответствует 1, а какому – 2 в признаке ``gender``) – это определите, посмотрев также на рост при разумном предположении, что в среднем мужчины выше.**

**<font color='red'>Варианты:</font>**
- 45530 женщин и 24470 мужчин
- 45530 мужчин и 24470 женщин
- 45470 женщин и 24530 мужчин
- 45470 мужчин и 24530 женщин

In [37]:
# Ваш код здесь
gender1_avg_height = df[df['gender']==1]['height'].mean()
gender2_avg_height = df[df['gender']==2]['height'].mean()

if gender1_avg_height > gender2_avg_height:
    m_gender = 1
    f_gender = 2
else:
    m_gender = 2
    f_gender = 1

m_num = len(df[df['gender']==m_gender])
f_num = len(df[df['gender']==f_gender])

if m_num > f_num:
    print('Ответ на вопрос №1: {} мужчин и {} женщин'.format(m_num, f_num))
else:
    print('Ответ на вопрос №1: {} женщин и {} мужчин'.format(f_num, m_num))

Ответ на вопрос №1: 45530 женщин и 24470 мужчин


**<font color='red'>Вопрос 2 (1 балл).</font> Кто в среднем чаще указывает, что употребляет алкоголь – мужчины или женщины?**

**<font color='red'>Варианты:</font>**
- мужчины 
- женщины

In [38]:
# Ваш код здесь
m_alco_num = len(df[(df['gender']==m_gender) & (df['alco']==1)])
f_alco_num = len(df[(df['gender']==f_gender) & (df['alco']==1)])

if m_alco_num > f_alco_num:
    print('Ответ на вопрос №2: мужчины')
else:
    print('Ответ на вопрос №2: женщины')

Ответ на вопрос №2: мужчины


**<font color='red'>Вопрос 3 (1 балл).</font> Во сколько раз (округленно, `round`) процент курящих среди мужчин больше, чем процент курящих среди женщин (по крайней мере, по этим анкетным данным)?**

**<font color='red'>Варианты:</font>**
- 4
- 8
- 12
- 16

In [39]:
# Ваш код здесь
m_smoke_num = len(df[(df['gender']==m_gender) & (df['smoke']==1)])
m_smoke_perc = m_smoke_num / m_num * 100

f_smoke_num = len(df[(df['gender']==f_gender) & (df['smoke']==1)])
f_smoke_perc = f_smoke_num / f_num * 100

print('Ответ на вопрос №3: {}'.format(round(m_smoke_perc / f_smoke_perc)))

Ответ на вопрос №3: 12


**<font color='red'>Вопрос 4 (1 балл).</font> Вы наверняка заметили, что значения возраста какие-то странные. Догадайтесь, в чем здесь измеряется возраст, и ответьте, на сколько месяцев (примерно) отличаются медианные значения возраста курящих и некурящих.**

In [ ]:
**<font color='red'>Варианты:</font>**
- 5
- 10
- 15
- 20

In [40]:
# Очевидно, что возраст измеряется в днях
print('Средний возраст в выборке в годах: {}'.format(round(df['age'].mean() / 365)))

Средний возраст в выборке в годах: 53


In [41]:
# Ваш код здесь
nonsmoke_median_age = df[df['smoke']==0]['age'].median()
smoke_median_age = df[df['smoke']==1]['age'].median()

age_diff_months = (nonsmoke_median_age - smoke_median_age) / 30
print('Ответ на вопрос №4: {}'.format(round(age_diff_months)))

Ответ на вопрос №4: 20


**<font color='red'>Вопрос 5 (2 балла).</font>** В [статье](https://ru.wikipedia.org/wiki/Сердечно-сосудистый_риск) на Википедии про сердечно-сосудистый риск показана шкала SCORE для расчёта риска смерти от сердечно-сосудистого заболевания в ближайшие 10 лет.

Давайте посмотрим на правый верхний прямоугольник, отображающий сегмент курящих мужчин в возрасте от 60 до 64 лет включительно. (Неочевидно, но тут для возраста и давления цифры означают верхнюю границу, и она не включается).

Видим 9-ку в левом нижнем углу этого прямоугольника и 47 – в правом верхнем. То есть если при этом систолическое (т.е. верхнее) артериальное давление – меньше 120 мм рт.ст., а уровень холестерина – 4 ммоль/л, то риск ССЗ оценивается примерно в 5 раз ниже, чем если бы давление лежало в интервале [160, 180), а холестерина было бы 8 ммоль/л.

Давайте посчитаем аналогичное значение, но на наших данных. 

Уточнения:
- Посчитайте признак ``age_years`` – возраст в годах, округлив до целых (`round`). Для данного примера отберите курящих мужчин от 60 до 64 лет включительно
- Категории уровня холестрина на рисунке и в наших данных отличаются. Отображение значений на картинке в значения признака ``cholesterol`` следующее: 4 ммоль/л $\rightarrow$ 1, 5-7 ммоль/л $\rightarrow$ 2, 8 ммоль/л $\rightarrow$ 3.
- Интересуют 2 подвыборки курящих мужчин возраста от 60 до 64 лет: первая с верхним артериальным давлением меньше 120 мм рт.ст. и концентрацией холестерина – 4 ммоль/л, а вторая – с верхним артериальным давлением от 160 до 180 мм рт.ст. (не включая верхнюю границу) и концентрацией холестерина – 8 ммоль/л.

**Во сколько раз (округленно, `round`) отличаются доли больных людей в этих двух подвыборках? Посчитайте на наших данных.**

**<font color='red'>Варианты:</font>**
- 2
- 3
- 4
- 5

In [42]:
# Ваш код здесь
df['age_years'] = round(df['age'] / 365)

sample1_sick_num = df[(df['age_years']>=60) & (df['age_years']<=64) & (df['smoke']==1) & 
                      (df['ap_hi']<120) & (df['cholesterol']==1)]['cardio'].sum()

sample1_num = len(df[(df['age_years']>=60) & (df['age_years']<=64) & (df['smoke']==1) & 
                      (df['ap_hi']<120) & (df['cholesterol']==1)])


sample2_sick_num = df[(df['age_years']>=60) & (df['age_years']<=64) & (df['smoke']==1) & 
                      (df['ap_hi']>=160) & (df['ap_hi']<180) &
                      (df['cholesterol']==3)]['cardio'].sum()

sample2_num = len(df[(df['age_years']>=60) & (df['age_years']<=64) & (df['smoke']==1) & 
                      (df['ap_hi']>=160) & (df['ap_hi']<180) &
                      (df['cholesterol']==3)])


sample1_sick_perc = sample1_sick_num / sample1_num
sample2_sick_perc = sample2_sick_num / sample2_num

print('Ответ на вопрос №5: {}'.format(round(sample2_sick_perc/sample1_sick_perc)))

Ответ на вопрос №5: 3


**<font color='red'>Вопрос 6 (2 балла).</font> Постройте новый признак – BMI ([Body Mass Index](https://en.wikipedia.org/wiki/Body_mass_index)). Для этого надо вес в килограммах поделить на квадрат роста в метрах. Нормальными считаются значения BMI от 18.5 до 25. Выберите верные утверждения.**

**<font color='red'>Утверждения:</font>**
- Медианный BMI по выборке лежит в пределах нормы
- У женщин в среднем BMI выше, чем у мужчин
- У здоровых в среднем BMI выше, чем у больных
- В сегменте здоровых и непьющих мужчин BMI ближе к норме, чем в сегменте здоровых и непьющих женщин

In [43]:
# Ваш код здесь
df['BMI'] = df['weight'] / (df['height']/100)**2

print('Ответы на вопрос №6:\n')

bmi_1 = df['BMI'].median()
print('- Медианный BMI по выборке лежит в пределах нормы: {}\n'.format((bmi_1>=18.5) & (bmi_1<=25)))

bmi_2_f = df[df['gender']==f_gender]['BMI'].mean()
bmi_2_m = df[df['gender']==m_gender]['BMI'].mean()
print('- У женщин в среднем BMI выше, чем у мужчин: {}\n'.format(bmi_2_f > bmi_2_m))

bmi_3_healthy = df[df['cardio']==0]['BMI'].mean()
bmi_3_sick = df[df['cardio']==1]['BMI'].mean()
print('- У здоровых в среднем BMI выше, чем у больных: {}\n'.format(bmi_3_healthy > bmi_3_sick))

bmi_4_m = df[(df['cardio']==0) & (df['alco']==0) & (df['gender']==m_gender)]['BMI'].mean()
bmi_4_f = df[(df['cardio']==0) & (df['alco']==0) & (df['gender']==f_gender)]['BMI'].mean()
print('Средний BMI в сегменте здоровых и непьющих мужчин: {:.2f}'.format(bmi_4_m))
print('Средний BMI в сегменте здоровых и непьющих женщин: {:.2f}'.format(bmi_4_f))
print('- В сегменте здоровых и непьющих мужчин BMI ближе к норме, чем в сегменте здоровых и непьющих женщин: {}'.\
     format(bmi_4_m < bmi_4_f))

Ответы на вопрос №6:

- Медианный BMI по выборке лежит в пределах нормы: False

- У женщин в среднем BMI выше, чем у мужчин: True

- У здоровых в среднем BMI выше, чем у больных: False

Средний BMI в сегменте здоровых и непьющих мужчин: 25.87
Средний BMI в сегменте здоровых и непьющих женщин: 26.85
- В сегменте здоровых и непьющих мужчин BMI ближе к норме, чем в сегменте здоровых и непьющих женщин: True


**<font color='red'>Вопрос 7 (2 балла).</font>** Можно заметить, что данные не особо-то чистые, много в них всякой "грязи" и неточностей. Еще лучше мы это увидим, когда обсудим визуализацию данных.

Отфильтруйте следующие сегменты пациентов (считаем это ошибками в данных)
- указанное нижнее значение артериального давления выше верхнего
- рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили (используйте `pd.Series.quantile`, если не знаете, что это такое – прочитайте)
- вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили

Этот вовсе не вся чистка данных, которую можно было проделать, но пока остановимся на этом.

**Сколько процентов данных (округленно, `round`) мы выбросили?**

**<font color='red'>Варианты:</font>**
- 8
- 9
- 10
- 11

In [44]:
# Ваш код здесь
dirty_df = df[(df['ap_lo']>df['ap_hi']) | 
              (df['height']<df['height'].quantile(q=0.025)) | 
              (df['height']>df['height'].quantile(q=0.975)) | 
              (df['weight']<df['weight'].quantile(q=0.025)) | 
              (df['weight']>df['weight'].quantile(q=0.975))]

print('Ответ на вопрос №7: {}'.format(round(len(dirty_df) / len(df) * 100)))

Ответ на вопрос №7: 10
